In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets



In [2]:
#dataframe:
df = pd.read_csv("students_drahi_production_consumption_hourly.csv")[10:]
df['datetime'] = pd.to_datetime(df['datetime'])
variable_names = df.columns[1:]  # Exclude the datetime column
df.head()

,datetime,AirTemp,pres,rain,rh,wd,ws,Global_Solar_Flux,Diffuse_Solar_Flux,Direct_Solar_Flux,...,kw_heatingcoolingtotal_zone1,kw_heatingcoolingtotal_zone2,kw_lights_zone1,kw_lights_zone2,kw_total_zone1,kw_total_zone2,kw_ventilation_zone1,kw_ventilation_zone2,kw_water_heater_zone2,plugs_zone2
10,2022-01-01 10:00:00+00:00,9.502,1003.658,0.0,86.584,164.683,2.487,225.399,61.658,649.858,...,-1.940,-2.293,-0.002,-0.239,2.225,2.588,-0.027,-0.043,-0.004,-0.156
11,2022-01-01 11:00:00+00:00,10.933,1003.570,0.0,79.723,166.577,2.109,282.286,100.783,593.404,...,-1.957,-2.249,-0.002,-0.296,2.243,2.606,-0.026,-0.043,-0.004,-0.156
12,2022-01-01 12:00:00+00:00,12.399,1002.691,0.0,72.151,175.309,2.586,325.197,86.113,759.032,...,-1.982,-2.282,-0.002,-0.238,2.269,2.917,-0.026,-0.043,-0.363,-0.157
13,2022-01-01 13:00:00+00:00,13.355,1002.551,0.0,66.119,223.277,2.893,178.243,98.997,274.745,...,-2.004,-2.273,-0.003,-0.238,2.292,2.572,-0.027,-0.043,-0.004,-0.155
14,2022-01-01 14:00:00+00:00,13.386,1001.755,0.0,65.928,196.903,2.869,161.048,73.627,400.527,...,-2.005,-2.303,-0.002,-0.249,2.295,2.606,-0.026,-0.043,-0.004,-0.156


In [3]:
#put energy consumption into positive values
columns_to_change = ['kw_lights_zone1', 
                     'kw_lights_zone2',
                     'kw_ventilation_zone1', 
                     'kw_ventilation_zone2', 
                     'kw_water_heater_zone2', 
                     'plugs_zone2', 
                     'kw_heater_corridor1_zone1',
                     'kw_heaters_corridor_zone2',
                     'kw_heaters_toilets_zone2',
                     'kw_heatingcoolingtotal_zone1',
                     'kw_heatingcoolingtotal_zone2']

df[columns_to_change] = -df[columns_to_change]

In [4]:
#specify the name for the legend in the plots
variable_names_mapping = {
    'kw_lights_zone2': 'Lights',
    'kw_ventilation_zone2': 'Ventilation',
    'kw_water_heater_zone2': 'Water Heater',
    'plugs_zone2': 'Plugs',
    'kw_heaters_corridor_zone2': 'Heaters Corridor',
    'kw_heaters_toilets_zone2': 'Heaters Toilets',
    'kw_heatingcoolingtotal_zone2': 'Heating/Cooling Total',
    'kw_total_zone2': 'Total Zone 2 consumption'
}

In [5]:


# Function to update the plot based on selected variables
def update_plot(selected_date, **selected_variables):
    selected_data = df[df['datetime'].dt.date == selected_date]
    plt.figure(figsize=(10, 6))

    for variable, value in selected_variables.items():
        if value:
            if variable in variable_names_mapping:
                label = variable_names_mapping[variable]
            else:
                label = variable
            plt.plot(selected_data['datetime'].dt.strftime('%Hh%M'), selected_data[variable], label=label)
    date = selected_date.strftime('%d/%m/%Y')
    plt.title(f'Energy consumption for Zone 2 on {date}')
    plt.xlabel('Time of the day')
    plt.ylabel('Energy consumption (kW)')
    plt.legend()

    n_xticks = 3
    xticks = selected_data['datetime'].dt.strftime('%Hh%M').index[::n_xticks] - selected_data['datetime'].dt.strftime('%Hh%M').index.min()   # Selecting every 5th index
    xticklabels = selected_data['datetime'].dt.strftime('%Hh%M').iloc[::n_xticks].values.tolist()    
    plt.xticks(xticks, xticklabels)
    #plt.xlim(selected_data['datetime'].dt.strftime('%Hh%M').min(), selected_data['datetime'].dt.strftime('%Hh%M').max())  # Set x-axis limits

    plt.show()


# Create checkboxes for each variable
variable_checkboxes = {variable: widgets.Checkbox(value=False, description=variable) for variable in variable_names}
# Create a slider with dates
date_slider = widgets.SelectionSlider(
    options=pd.to_datetime(df['datetime'].dt.date.unique()),
    value=pd.to_datetime(df['datetime'].dt.date.min()),
    description='Select Date',
    continuous_update=False
)
# Create the interactive plot
interact(update_plot, selected_date=date_slider, **variable_checkboxes)

# Show the plot
plt.show()

interactive(children=(SelectionSlider(continuous_update=False, description='Select Date', options=(Timestamp('…

### Evaluer le chauffage d'un bureau et calculer l'économie hebdomadaire